In [1]:
# Import
%matplotlib inline

# Start Python Imports
import math, time, random, datetime

# Data Manipulation
import numpy as np
import pandas as pd

# Visualization 
import matplotlib.pyplot as plt
import missingno
import seaborn as sns
plt.style.use('seaborn-whitegrid')

# Preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize

# Machine learning
import catboost
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier, Pool, cv
# import XGBoost

# # Let's be rebels and ignore warnings for now
# import warnings
# warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("high_diamond_ranked_10min.csv")
df.head()

,gameId,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,...,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
0,4519157822,0,28,2,1,9,6,11,0,0,...,0,16567,6.8,17047,197,55,-643,8,19.7,1656.7
1,4523371949,0,12,1,0,5,5,5,0,0,...,1,17620,6.8,17438,240,52,2908,1173,24.0,1762.0
2,4521474530,0,15,0,0,7,11,4,1,1,...,0,17285,6.8,17254,203,28,1172,1033,20.3,1728.5
3,4524384067,0,43,1,0,4,5,5,1,0,...,0,16478,7.0,17961,235,47,1321,7,23.5,1647.8
4,4436033771,0,75,4,0,6,6,6,0,0,...,0,17404,7.0,18313,225,67,1004,-230,22.5,1740.4


In [4]:
df.drop("gameId", axis=1, inplace=True)

In [5]:
clean_df = df.copy()
dropcols = ['blueWardsPlaced', 'blueWardsDestroyed', 'redWardsPlaced', 'redWardsDestroyed',
           'blueFirstBlood', 'redFirstBlood', 'redTotalJungleMinionsKilled', 'redTotalMinionsKilled', 'blueTotalJungleMinionsKilled', 'blueTotalMinionsKilled']

clean_df = clean_df.drop(dropcols, axis=1)

In [6]:
clean_df.head()

,blueWins,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueHeralds,blueTowersDestroyed,blueTotalGold,blueAvgLevel,...,redDragons,redHeralds,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
0,0,9,6,11,0,0,0,0,17210,6.6,...,0,0,0,16567,6.8,17047,-643,8,19.7,1656.7
1,0,5,5,5,0,0,0,0,14712,6.6,...,1,1,1,17620,6.8,17438,2908,1173,24.0,1762.0
2,0,7,11,4,1,1,0,0,16113,6.4,...,0,0,0,17285,6.8,17254,1172,1033,20.3,1728.5
3,0,4,5,5,1,0,1,0,15157,7.0,...,0,0,0,16478,7.0,17961,1321,7,23.5,1647.8
4,0,6,6,6,0,0,0,0,16400,7.0,...,1,0,0,17404,7.0,18313,1004,-230,22.5,1740.4


In [7]:
correlations = clean_df[clean_df.columns[1:]].apply(lambda x: x.corr(clean_df['blueWins']))
cols = []
for col in correlations.index:
    if (correlations[col]>0.3 or correlations[col]<-0.3):
        cols.append(col)
cols

['blueKills',
 'blueDeaths',
 'blueTotalGold',
 'blueAvgLevel',
 'blueTotalExperience',
 'blueGoldDiff',
 'blueExperienceDiff',
 'blueGoldPerMin',
 'redKills',
 'redDeaths',
 'redTotalGold',
 'redAvgLevel',
 'redTotalExperience',
 'redGoldDiff',
 'redExperienceDiff',
 'redGoldPerMin']

In [8]:
clean_df = clean_df[cols]
clean_df.head()

,blueKills,blueDeaths,blueTotalGold,blueAvgLevel,blueTotalExperience,blueGoldDiff,blueExperienceDiff,blueGoldPerMin,redKills,redDeaths,redTotalGold,redAvgLevel,redTotalExperience,redGoldDiff,redExperienceDiff,redGoldPerMin
0,9,6,17210,6.6,17039,643,-8,1721.0,6,9,16567,6.8,17047,-643,8,1656.7
1,5,5,14712,6.6,16265,-2908,-1173,1471.2,5,5,17620,6.8,17438,2908,1173,1762.0
2,7,11,16113,6.4,16221,-1172,-1033,1611.3,11,7,17285,6.8,17254,1172,1033,1728.5
3,4,5,15157,7.0,17954,-1321,-7,1515.7,5,4,16478,7.0,17961,1321,7,1647.8
4,6,6,16400,7.0,18543,-1004,230,1640.0,6,6,17404,7.0,18313,1004,-230,1740.4


In [15]:
from sklearn.preprocessing import MinMaxScaler
X = clean_df
y = df["blueWins"]

scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Logistic Regression

In [17]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


lr = LogisticRegression(solver='lbfgs')
lr.fit(X_train, y_train)
pred_lr = lr.predict(X_test)
acc_lr = accuracy_score(pred_lr, y_test)
acc_lr

0.7257085020242915

# Random Forest

In [107]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV


rf = RandomForestClassifier()
rf.fit(X_train, y_train)

predrf = rf.predict(X_test)
accrf = accuracy_score(predrf, y_test)
accrf

C:\Users\karth\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.6916329284750338

# SVC

# Gradient Boosting

In [103]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV


gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)

predrf = gb.predict(X_test)
accrf = accuracy_score(predrf, y_test)
accrf

0.7223346828609987

# XGBClassifier

# GaussianNB

In [18]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

nb = GaussianNB()
nb.fit(X_train, y_train)
prednb = nb.predict(X_test)
accnb = accuracy_score(prednb, y_test)
accnb

0.7246963562753036